<a href="https://colab.research.google.com/github/yueguo1997/Group4_Low_Demensional_Representation_with_Big_Genomics_Data/blob/step45/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Package

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

import os
cur_path = "/content/drive/MyDrive/Bigdataproject/"
os.chdir(cur_path)
!pwd

/content/drive/MyDrive/Bigdataproject


In [3]:
!pip install pyspark
!pip install pyspark[sql]
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 19.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=f4e2b8ccdfbd147e7e94e9509446323deda6698f04ebbf9247cab0a6b4abd400
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns

In [4]:
import pyspark as spark
import time
from operator import add
from pyspark.sql import SparkSession

# Step 0: Read Data

## Set spark session

In [5]:
# set spark session
ss = (SparkSession
  .builder
  .master("local[5]")
  .appName("fqproject")
  .getOrCreate())


## Set spark context

In [6]:
# set spark session
sc = (spark
  .SparkContext
  .getOrCreate(spark
    .SparkConf()
    .setAppName("fqproject")
    .setMaster('local[*]')
    .set('spark.executor.memory', '4G')
    .set('spark.driver.memory', '4G')
    .set('spark.driver.maxResultSize', '4G')))

## Read reads.fq

In [7]:
# read reads.fq and extract the second line of every four lines.
reads = (sc
  .textFile(cur_path + "data/reads.fq")
  .zipWithIndex()
  .filter(lambda x: (x[1]+1)%2==0 and (x[1]+1)%4!=0)
  .map(lambda x: x[0]))

In [8]:
reads.take(10)

['TCCTTACTGGTTTTGCAGGTAACTTATAGAGTATTTCCACTTCCCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATC',
 'GGTTTTTCAGGTAACTTATAGAGTATTTCCACTTCCCTTCTCCTATCCCTGGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATA',
 'AGGTAACTTATAGAGTACTTCCACTTCCCTTCTCCTATCCCTTGAAAAATTGTCATTGATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTG',
 'CACTTCCCTTCTACTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATG',
 'CCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCCCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTTTT',
 'ATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCGTGCCTATTAGATTCATT',
 'TCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGAATATAAAAAAATTTTATTTTATT',
 'TATCCATATGGCATAATCAAAGAATCAATTGTTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGAATATAAAAAAATTTTATTTTATTTTC',
 'TCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTATTAGATTCATTTAGATTATAAAAAAATTTTATTTTATTTTCACTTATTTCTTCTCCA',
 'TGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTACTAGATTCATTTAG

## Read reference.fa

In [9]:
# read reference
reference = (ss
  .read
  .csv(
      cur_path + "data/reference_chr21_20000000_20050000.fa",
      inferSchema=True,
      header=True))

In [10]:
reference.show(10)

+--------+--------+--------------------+
|   start|     end|            sequence|
+--------+--------+--------------------+
|20000000|20000100|CCCTTCTCCTATCCCTT...|
|20000100|20000200|TAGATTCATTTAGAATA...|
|20000200|20000300|TTCTTCTTCCTGAAGAA...|
|20000300|20000400|ACTTTTCAAGGATAGTT...|
|20000400|20000500|CTGACAGGACTTCTGCC...|
|20000500|20000600|TTTCCTTTTTTTTTTCT...|
|20000600|20000700|ATTATAAAAAGGGAGGG...|
|20000700|20000800|TTCTTTTCTTTTTCTCC...|
|20000800|20000900|ATAAATTTCTGCTTGAA...|
|20000900|20001000|TTCGTTAGTGTTTTTTA...|
+--------+--------+--------------------+
only showing top 10 rows



In [11]:
reference.count()

500

# Step 1: Cut reads into kmers = 15

## Define K-mers

In [12]:
# input a string s and cut k-mers
def extract_kmers(s: str, k: int = 15) -> list:
    return [s[i:i+k] for i in range(len(s)-k+1)]

## Map K-mers to reads

In [13]:
# K-mers of reads
reads_kmers = (reads
  .map(extract_kmers))

In [14]:
reads_kmers.take(2)

[['TCCTTACTGGTTTTG',
  'CCTTACTGGTTTTGC',
  'CTTACTGGTTTTGCA',
  'TTACTGGTTTTGCAG',
  'TACTGGTTTTGCAGG',
  'ACTGGTTTTGCAGGT',
  'CTGGTTTTGCAGGTA',
  'TGGTTTTGCAGGTAA',
  'GGTTTTGCAGGTAAC',
  'GTTTTGCAGGTAACT',
  'TTTTGCAGGTAACTT',
  'TTTGCAGGTAACTTA',
  'TTGCAGGTAACTTAT',
  'TGCAGGTAACTTATA',
  'GCAGGTAACTTATAG',
  'CAGGTAACTTATAGA',
  'AGGTAACTTATAGAG',
  'GGTAACTTATAGAGT',
  'GTAACTTATAGAGTA',
  'TAACTTATAGAGTAT',
  'AACTTATAGAGTATT',
  'ACTTATAGAGTATTT',
  'CTTATAGAGTATTTC',
  'TTATAGAGTATTTCC',
  'TATAGAGTATTTCCA',
  'ATAGAGTATTTCCAC',
  'TAGAGTATTTCCACT',
  'AGAGTATTTCCACTT',
  'GAGTATTTCCACTTC',
  'AGTATTTCCACTTCC',
  'GTATTTCCACTTCCC',
  'TATTTCCACTTCCCT',
  'ATTTCCACTTCCCTT',
  'TTTCCACTTCCCTTC',
  'TTCCACTTCCCTTCT',
  'TCCACTTCCCTTCTC',
  'CCACTTCCCTTCTCC',
  'CACTTCCCTTCTCCT',
  'ACTTCCCTTCTCCTA',
  'CTTCCCTTCTCCTAT',
  'TTCCCTTCTCCTATC',
  'TCCCTTCTCCTATCC',
  'CCCTTCTCCTATCCC',
  'CCTTCTCCTATCCCT',
  'CTTCTCCTATCCCTT',
  'TTCTCCTATCCCTTG',
  'TCTCCTATCCCTTGA',
  'CTCCTATCCC

# Step 2: • Cut reference bins into kmers = 15

In [15]:
# K-mers of reference
reference_kmers = (reference
  .select('sequence')
  .rdd
  .flatMap(lambda x: x)
  .map(extract_kmers))

In [16]:
reference_kmers.take(2)

[['CCCTTCTCCTATCCC',
  'CCTTCTCCTATCCCT',
  'CTTCTCCTATCCCTT',
  'TTCTCCTATCCCTTG',
  'TCTCCTATCCCTTGA',
  'CTCCTATCCCTTGAA',
  'TCCTATCCCTTGAAA',
  'CCTATCCCTTGAAAA',
  'CTATCCCTTGAAAAA',
  'TATCCCTTGAAAAAT',
  'ATCCCTTGAAAAATT',
  'TCCCTTGAAAAATTG',
  'CCCTTGAAAAATTGT',
  'CCTTGAAAAATTGTC',
  'CTTGAAAAATTGTCA',
  'TTGAAAAATTGTCAT',
  'TGAAAAATTGTCATT',
  'GAAAAATTGTCATTT',
  'AAAAATTGTCATTTA',
  'AAAATTGTCATTTAT',
  'AAATTGTCATTTATT',
  'AATTGTCATTTATTT',
  'ATTGTCATTTATTTC',
  'TTGTCATTTATTTCT',
  'TGTCATTTATTTCTC',
  'GTCATTTATTTCTCT',
  'TCATTTATTTCTCTT',
  'CATTTATTTCTCTTA',
  'ATTTATTTCTCTTAT',
  'TTTATTTCTCTTATC',
  'TTATTTCTCTTATCC',
  'TATTTCTCTTATCCA',
  'ATTTCTCTTATCCAT',
  'TTTCTCTTATCCATA',
  'TTCTCTTATCCATAT',
  'TCTCTTATCCATATG',
  'CTCTTATCCATATGG',
  'TCTTATCCATATGGC',
  'CTTATCCATATGGCA',
  'TTATCCATATGGCAT',
  'TATCCATATGGCATA',
  'ATCCATATGGCATAA',
  'TCCATATGGCATAAT',
  'CCATATGGCATAATC',
  'CATATGGCATAATCA',
  'ATATGGCATAATCAA',
  'TATGGCATAATCAAA',
  'ATGGCATAAT

# Step 3: Collect all kmers and build a distinct kmer set

• Hint: you can use python “set” function

• Report the number of distinct kmers (N)

In [17]:
# Merge and distinct, collect to a set
kmers = set(reads_kmers 
  .union(reference_kmers)
  .flatMap(lambda x: x) 
  .distinct()
  .collect())

In [18]:
len(kmers)

72530

**The number of distinct kmers is 72,530**


In [19]:
reads

PythonRDD[40] at RDD at PythonRDD.scala:53

# Step4

In [20]:
from pyspark.ml.linalg import Vectors
import pandas as pd

In [21]:

# turn reads to list 
r_list = reads.collect()

In [22]:
# check feature vector of every kmer in each read
def check_kmer_features(kmer):
  m = []
  

  for i in range(len(r_list)):
    if kmer in r_list[i]:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (kmer,Vectors.dense(Vectors.sparse(len(r_list),m,v)))

In [23]:
kmers_rdd = sc.parallelize(list(kmers))

In [24]:
reads_features = kmers_rdd.map(lambda x: check_kmer_features(x))

In [25]:
reads_features_df = reads_features.toDF(['kmer','read_features'])

In [26]:
#generate the dataframe with sparse vector
reads_features_df.show(1)

+---------------+--------------------+
|           kmer|       read_features|
+---------------+--------------------+
|GATCTGATGGTTTTA|[0.0,0.0,0.0,0.0,...|
+---------------+--------------------+
only showing top 1 row



In [27]:
from pyspark.sql.functions import col
from pyspark.ml.functions import vector_to_array

In [28]:
reads_features_df = reads_features_df.withColumn("read", vector_to_array("read_features")).select(["kmer"] + [col("read")[i] for i in range(2000)])

In [29]:
read_name = {r_list[i]: 'reads' + str(i) for i in range(len(r_list))}

In [30]:
# old code
def check_kmer_features_old1(read):
  m = []
  kmers_list = list(kmers)
  for i in range(len(kmers_list)):
    if kmers_list[i] in read:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (read_name[read],Vectors.sparse(len(kmers_list),m,v))


In [31]:
reads_features = reads.map(lambda x: check_kmer_features_old1(x))

In [32]:
reads_features.take(2)

[('reads0',
  SparseVector(72530, {171: 1.0, 853: 1.0, 1693: 1.0, 2458: 1.0, 4005: 1.0, 4616: 1.0, 4888: 1.0, 6088: 1.0, 6581: 1.0, 7639: 1.0, 7932: 1.0, 9006: 1.0, 11636: 1.0, 13188: 1.0, 16374: 1.0, 17639: 1.0, 18175: 1.0, 18275: 1.0, 19241: 1.0, 19545: 1.0, 20315: 1.0, 20390: 1.0, 21783: 1.0, 22082: 1.0, 23392: 1.0, 23736: 1.0, 23934: 1.0, 23957: 1.0, 24718: 1.0, 25523: 1.0, 25944: 1.0, 26522: 1.0, 26768: 1.0, 27978: 1.0, 28197: 1.0, 30096: 1.0, 31463: 1.0, 34048: 1.0, 34659: 1.0, 35092: 1.0, 36089: 1.0, 37854: 1.0, 38460: 1.0, 40048: 1.0, 40474: 1.0, 40766: 1.0, 41118: 1.0, 41690: 1.0, 42496: 1.0, 42568: 1.0, 42911: 1.0, 43064: 1.0, 43484: 1.0, 44215: 1.0, 45098: 1.0, 46243: 1.0, 50536: 1.0, 51935: 1.0, 53798: 1.0, 54048: 1.0, 54198: 1.0, 54916: 1.0, 55896: 1.0, 56222: 1.0, 58227: 1.0, 59434: 1.0, 59625: 1.0, 61056: 1.0, 61065: 1.0, 61218: 1.0, 61340: 1.0, 61473: 1.0, 61891: 1.0, 61979: 1.0, 62585: 1.0, 63353: 1.0, 63443: 1.0, 65006: 1.0, 65355: 1.0, 65728: 1.0, 66313: 1.0, 67215: 

# Step5

In [33]:
reference_rdd = reference.select("sequence").rdd.map(lambda x: x['sequence'])

In [34]:
reference_list = reference_rdd.collect()
reference_list[0]

'CCCTTCTCCTATCCCTTGAAAAATTGTCATTTATTTCTCTTATCCATATGGCATAATCAAAGAATAAATTGGTGATATTTGTTCAAAAATCCATGCCTAT'

In [35]:
#generate the name of columns
start_rdd = reference.select("start").rdd.map(lambda x: x['start'])
end_rdd = reference.select("end").rdd.map(lambda x: x['end'])
s_list = start_rdd.collect()
e_list = end_rdd.collect()
reference_name = ['bin' + str(s_list[i]) + '_' +  str(e_list[i]) for i in range(len(s_list))]
reference_name[0]

'bin20000000_20000100'

In [36]:
# check feature vector of every kmer in each reference
def check_kmer_read_features_sequence(kmer):
  m = []
  for i in range(len(reference_list)):
    if kmer in reference_list[i]:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (kmer,Vectors.dense(Vectors.sparse(len(reference_list),m,v)))

In [37]:
reference_features = kmers_rdd.map(lambda x:check_kmer_read_features_sequence(x))

In [38]:
# turn to dataframe
reference_features_df = reference_features.toDF(['kmer','reference_features'])

In [39]:
reference_features_df.show(1) 

+---------------+--------------------+
|           kmer|  reference_features|
+---------------+--------------------+
|GATCTGATGGTTTTA|[0.0,0.0,0.0,0.0,...|
+---------------+--------------------+
only showing top 1 row



In [40]:
# expand the dense vector to different collomns
reference_features_df = reference_features_df.withColumn("xs", vector_to_array("reference_features")).select(["kmer"] + [col("xs")[i] for i in range(len(reference_name))])

In [41]:
reference_name = ['kmer'] + reference_name
reference_features_df = reference_features_df.select([col(c).alias(reference_name[i]) for i, c in enumerate(reference_features_df.columns)])

In [42]:
reference_name_d = {reference_list[i]: reference_name[i+1] for i in range(len(reference_list))}
len(reference_name_d)

500

In [43]:
# old code
def check_kmer_features_old(read):
  m = []
  kmers_list = list(kmers)
  for i in range(len(kmers_list)):
    if kmers_list[i] in read:
      m.append(i)
  v = [1.0 for i in range(len(m))]
  return (reference_name_d[read],Vectors.sparse(len(kmers_list),m,v))

In [44]:
# old code 
reference_features = reference_rdd.map(lambda x: check_kmer_features_old(x))
reference_features.take(1)

[('bin20000000_20000100',
  SparseVector(72530, {171: 1.0, 853: 1.0, 2585: 1.0, 4998: 1.0, 6088: 1.0, 7344: 1.0, 7639: 1.0, 8859: 1.0, 9006: 1.0, 9267: 1.0, 11636: 1.0, 13473: 1.0, 13825: 1.0, 14089: 1.0, 15244: 1.0, 15320: 1.0, 15828: 1.0, 16306: 1.0, 16656: 1.0, 17210: 1.0, 17639: 1.0, 18175: 1.0, 19241: 1.0, 19545: 1.0, 20315: 1.0, 20390: 1.0, 21783: 1.0, 22082: 1.0, 23006: 1.0, 25523: 1.0, 25619: 1.0, 26028: 1.0, 26522: 1.0, 27978: 1.0, 33448: 1.0, 33757: 1.0, 34048: 1.0, 34649: 1.0, 36127: 1.0, 38460: 1.0, 38726: 1.0, 40474: 1.0, 40766: 1.0, 41118: 1.0, 41690: 1.0, 42046: 1.0, 42496: 1.0, 42568: 1.0, 43064: 1.0, 43791: 1.0, 44215: 1.0, 45098: 1.0, 46243: 1.0, 46877: 1.0, 47323: 1.0, 48119: 1.0, 49411: 1.0, 49877: 1.0, 50170: 1.0, 51935: 1.0, 53099: 1.0, 53763: 1.0, 53798: 1.0, 54048: 1.0, 54098: 1.0, 57831: 1.0, 58227: 1.0, 58708: 1.0, 59434: 1.0, 61056: 1.0, 61218: 1.0, 61979: 1.0, 62472: 1.0, 62585: 1.0, 63353: 1.0, 63443: 1.0, 65035: 1.0, 65355: 1.0, 66313: 1.0, 67543: 1.0, 675


# Step 6 （没写完版orz）

In [45]:
import random
import numpy as np
from pyspark.sql.functions import rand
from pyspark.ml.feature import MinHashLSH

In [46]:
read_df_new = reads_features.toDF(['read','feature'])
read_df_new.show(1)
                                  
                                  

+------+--------------------+
|  read|             feature|
+------+--------------------+
|reads0|(72530,[171,853,1...|
+------+--------------------+
only showing top 1 row



In [47]:
mh = MinHashLSH(inputCol="feature", outputCol="hashes", numHashTables=1000)
model = mh.fit(read_df_new)

In [48]:
reference_df_new = reference_features.toDF(['reference','feature'])

In [60]:
model.transform(read_df_new).show(1)

+-------+--------------------+--------------------+
|   read|             feature|              hashes|
+-------+--------------------+--------------------+
| reads0|(72530,[171,853,1...|[[2.3998118E7], [...|
| reads1|(72530,[171,853,2...|[[1.568162E7], [2...|
| reads2|(72530,[3171,4521...|[[1.568162E7], [1...|
| reads3|(72530,[171,853,2...|[[1.568162E7], [1...|
| reads4|(72530,[171,1045,...|[[1.568162E7], [1...|
| reads5|(72530,[171,853,2...|[[1.568162E7], [9...|
| reads6|(72530,[146,322,5...|[[1.568162E7], [2...|
| reads7|(72530,[146,322,5...|[[1.568162E7], [2...|
| reads8|(72530,[667,1127,...|[[2.2829075E7], [...|
| reads9|(72530,[146,322,5...|[[5.251896E7], [2...|
|reads10|(72530,[146,322,7...|[[4.7921676E7], [...|
|reads11|(72530,[68,336,66...|[[2679040.0], [62...|
|reads12|(72530,[146,751,2...|[[2.0791872E7], [...|
|reads13|(72530,[1640,2438...|[[9264173.0], [5....|
|reads14|(72530,[2438,4432...|[[9264173.0], [6....|
|reads15|(72530,[2438,3332...|[[5322964.0], [3....|
|reads16|(72

In [49]:
finalresult = model.approxSimilarityJoin(read_df_new, reference_df_new, 1, distCol="JaccardDistance")\
    .select(col("datasetA.read").alias("idread"),
            col("datasetB.reference").alias("idreference"),
            col("JaccardDistance"))

In [50]:
finalresult.show(1)

+---------+--------------------+-------------------+
|   idread|         idreference|    JaccardDistance|
+---------+--------------------+-------------------+
|reads1117|bin20025100_20025200|0.02298850574712641|
+---------+--------------------+-------------------+
only showing top 1 row



In [54]:
df_grouped = finalresult.groupBy("idread").min("JaccardDistance")

In [55]:
df_grouped.show(1)

+---------+--------------------+
|   idread|min(JaccardDistance)|
+---------+--------------------+
|reads1102|  0.6666666666666667|
+---------+--------------------+
only showing top 1 row



SyntaxError: ignored

In [ ]:
final = result_df.select('idread','idreference','JaccardDistance')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-78-1847c6d93c24>", line 1, in <cell line: 1>
    final = result_df.select('idread','idreference','JaccardDistance')
NameError: name 'result_df' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local

## read signature

## reference signature